# DRMM (Deep Relevance Matching Model)

_Ismaël Bonneau_


But de ce notebook: Construire une architecture DRMM fonctionnelle avec Keras.

Pour cela, 2 étapes:

- construire la chaîne de pré traitements:
    - générer des paires document-requête non pertinentes et pertinentes pour l'apprentissage
    - générer des histogrammes d'interaction locales au niveau document-requête
- construire l'architecture DRMM

Les interractions sont pour le moment des interactions locales sur des word embeddings et sont mesurées comme une similarité cosinus entre les vecteurs des mots de la requête et ceux du document.

In [10]:
from gensim.models import KeyedVectors
import numpy as np
import matplotlib.pyplot as plt
from os import sep
import os

%matplotlib inline

embeddings_path = "embeddings_wiki2017"
dataset_path = "data"

## Pré traitements: 

### Récupérer des word embeddings 

Ce word embedding a les caractéristiques suivantes:

- Gensim Continuous Skipgram
- taille de vecteur ${300}$
- window ${5}$
- entrainé sur wikipédia février 2017 en langue anglaise
- lemmatisation
- ${273992}$ mots

http://vectors.nlpl.eu/repository/

In [190]:
model = KeyedVectors.load_word2vec_format(embeddings_path + sep + "model.bin", binary=True)

In [191]:
vocabulary = [w for w in model.vocab]

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='word', vocabulary=vocabulary, binary=True, lowercase=False)

In [197]:
model.most_similar("Toulouse")

[('Montpellier', 0.7763358354568481),
 ('Rennes', 0.7535181045532227),
 ('Nantes', 0.7404719591140747),
 ('Marseille', 0.7251037359237671),
 ('nîmes', 0.7154472470283508),
 ('Narbonne', 0.7153515815734863),
 ('Béziers', 0.7148354649543762),
 ('Poitiers', 0.7118483781814575),
 ('Perpignan', 0.7109518051147461),
 ('Bordeaux', 0.7101389765739441)]

### On Récupère les paires de pertinence/non pertinence pour chaque requête 

On génère un dictionnaire qui contient pour chaque requête en clé, un dictionnaire contenant 2 listes:
- "relevant" contenant des id de document pertinents pour la requête.
- "irrelevant" contenant des id de document non pertinents pour la requête.

In [3]:
paires = {}

with open(dataset_path + sep + "qrels.robust2004.txt", "r") as f:
    for line in f:
        lol = line.split()
        paires.setdefault(lol[0], {})
        paires[lol[0]].setdefault('relevant', []) 
        paires[lol[0]].setdefault('irrelevant', [])
        if lol[-1] == '1':
            paires[lol[0]]["relevant"].append(lol[2])
        else:
            paires[lol[0]]["irrelevant"].append(lol[2])

### On récupère les requêtes:

Elles se trouvent sous forme de tuple ([mots clés], [texte de la requête]). On ne garde que les mots clés.

In [202]:
import ast
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation

def clean(txt):
    return txt.replace(",", "").replace(".", "")

with open(dataset_path + sep + "robust2004.txt", "r") as f:
    queries = ast.literal_eval(f.read())
queries = {d:clean(queries[d][0]) for d in queries}

In [204]:
print(queries["301"])
print(queries["401"])

international organized crime
foreign minorities germany


In [7]:
def query_term_maxlen(queries):
    return np.max([len(queries[q].split()) for q in queries])

### Le DRMM a deux entrées: une entrée interactions et une entrée termes.

L'entrée termes prend un vecteur d'idf des termes de la requête. Il faut donc pouvoir récupérer efficacement des idf. Pour cela, on construit un dictionnaire terme -> idf qui nous servira dans l'étape d'après.

In [ ]:
# J'essaie le chargement des idf avec sklearn mais tu peux essayer avec elasticsearch'

from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [] #balancer l'ouverture des docs ici
vectorizer = TfidfVectorizer(
                        use_idf=True,
                        smooth_idf=True, 
                        sublinear_tf=False,
                        binary=False,
                        min_df=1, max_df=1.0, max_features=None,
                        ngram_range=(1,1))

X = vectorizer.fit(corpus) #osef du transform
idf = vectorizer.idf_
idf = dict(zip(vectorizer.get_feature_names(), idf)) #notre dictionnaire terme -> idf

import pickle

pickle.dump(idf, open("idf_robust2004.pkl", "wb"))

### On peut maintenant construire l'histogramme des interactions entre les embeddings de la requête et ceux du document.

On prend comme exemple 4 bins: ${[-1, -0.5]}$ ${[-0.5, 0]}$ ${[0, 0.5]}$ ${[0.5, 1]}$.

**Plusieurs manières de construire un histogramme**: compter le nombre de valeurs, compter puis normaliser...

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import ast

class DRMMinputGenerator:
    """
    
    """
    
    def __init__(self, query_term_maxlen, intervals, normalize=False):
        self.q_max_len = query_term_maxlen
        self.intervals = intervals
        self.normalize = normalize
        self.intvlsArray = np.linspace(-1, 1, self.intervals)
        
    def set_params(self, model_wv, vectorizer):
        self.model_wv = model_wv
        self.vectorizer = vectorizer
    
    def get_idf_vec(self, q):
        """
        """
        vec = np.zeros(self.q_max_len)
        for i, term in enumerate(q.split()):
            if term in self.model_wv: #je suis pas sûr de mon coup là
                vec[i] = self.idf_values[term] 
                
        return vec
        
    def hist(self, query, document):
        """
        """
        X = []
        for i in query.nonzero()[1]:
            histo = []
            for j in document.nonzero()[1]:
                histo.append(cosine_similarity([self.model_wv.vectors[i]], [self.model_wv.vectors[j]])[0][0])
            histo, bin_edges = np.histogram(histo, bins=self.intvlsArray)
            if self.normalize:
                histo = histo / histo.sum()
            X.append(histo)
        if len(query.nonzero()[1]) < self.query_term_maxlen:
            # compléter avec des zéro
            for i in range(self.query_term_maxlen - len(query.nonzero()[1])):
                X.append([0]*self.intervals)
        #retourner histogramme interactions
        return np.array(X)
    
    def load_data(self, queries_file, qrel_file, idf, test_size=0.2):
        """
        """
        self.idf_values = idf
        #lecture des requêtes
        with open(dataset_path + sep + queries_file, "r") as f:
            queries = ast.literal_eval(f.read())
            queries = {d:queries[d][0] for d in queries}
        #spliter les requêtes en train/test
        lol = list(queries.keys())
        random.shuffle(lol)
        test_keys = lol[:int(test_size * len(lol))]
        train_keys = lol[int(test_size * len(lol)):]
        
        #maintenant on crée les paires
        paires = {}

        with open(dataset_path + sep + qrel_file, "r") as f:
            for line in f:
                lol = line.split()
                paires.setdefault(lol[0], {})
                paires[lol[0]].setdefault('relevant', []) 
                paires[lol[0]].setdefault('irrelevant', [])
                if lol[-1] == '1':
                    paires[lol[0]]["relevant"].append(lol[2])
                else:
                    paires[lol[0]]["irrelevant"].append(lol[2])
                    
        #pour chaque requête on va générer autant de paires relevant que irrelevant
        #pour nos besoins on va alterner paires positives et paires négatives
        train_hist = [] # les histogrammes d'interraction
        test_hist = []
        train_idf = [] #les vecteurs d'idf
        test_idf = []
        
        for requete in train_keys:
            #recuperer les mots dont on connait les embeddings dans la query
            q = self.vectoriser.transform(queries[requete])
            idf_vec = self.get_idf_vec(queries[requete])
            for pos, neg in zip(paires[requete]["relevant"], paires[requete]["irrelevant"]):
                #lire le doc, la requete et creer l'histogramme d'interraction
                
                #yanis fous la partie lecture de doc ici stp
                d = self.vectoriser.transform("mettre le doc positif ici stp")
                train_hist.append(self.hist(q, d)) #append le doc positif
                train_idf.apprend(idf_vec) #append le vecteur idf de la requête
                d = self.vectoriser.transform("mettre le doc negatif ici stp")
                train_hist.append(self.hist(q, d)) #append le doc négatif
                train_idf.apprend(idf_vec) #append le vecteur idf de la requête
        train_labels = np.zeros(len(train_hist))
        train_labels[::2] = 1
        
        
        for requete in test_keys:
            #recuperer les mots dont on connait les embeddings dans la query
            q = self.vectoriser.transform(queries[requete])
            idf_vec = self.get_idf_vec(queries[requete])
            for pos, neg in zip(paires[requete]["relevant"], paires[requete]["irrelevant"]):
                #lire le doc, la requete et creer l'histogramme d'interraction
                
                #yanis fous la partie lecture de doc ici stp
                d = self.vectoriser.transform("mettre le doc positif ici stp")
                test_hist.append(self.hist(q, d)) #append le doc positif
                test_idf.apprend(idf_vec) #append le vecteur idf de la requête
                d = self.vectoriser.transform("mettre le doc negatif ici stp")
                test_hist.append(self.hist(q, d)) #append le doc négatif
                test_idf.apprend(idf_vec) #append le vecteur idf de la requête
        test_labels = np.zeros(len(train_hist))
        test_labels[::2] = 1
        
        #éventuellement sauvegarder tout ça sur le disque comme ça c fait une bonne fois pour toutes...

## Architecture du modèle

D'après <a href="https://dl.acm.org/citation.cfm?id=2983769">l'article</a>

Code d'après <a href="https://github.com/sebastian-hofstaetter/neural-ranking-drmm/blob/master/neural-ranking/keras_model.py">ce génie</a>

In [88]:
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Dense, Activation, Lambda, Permute, merge
from keras.layers import Reshape, Dot
from keras.activations import softmax


def build_keras_model(params):
    """
    """
    
    initializer_interactions = keras.initializers.RandomUniform(minval=-0.1, maxval=0.1, seed=11)
    initializer_gating = keras.initializers.RandomUniform(minval=-0.01, maxval=0.01, seed=11)
    
    #input interactions
    interactions = Input(name='interactions', shape=(params['query_term_maxlen'], params['hist_size']))
    
    #input des term vectors de la query
    query = Input(name='term_vector', shape=(params['query_term_maxlen'],1))

    #partie feed forward
    z = interactions
    for i in range(len(params['hidden_sizes'])):
        z = Dense(params['hidden_sizes'][i], kernel_initializer=initializer_interactions, name="dense_layer_{}".format(i))(z)
        z = Activation('tanh', name="activation_of_layer_{}".format(i))(z)

    z = Permute((2, 1))(z)
    z = Reshape((params['query_term_maxlen'],))(z)

    #la partie term gating
    q_w = Dense(1, kernel_initializer=initializer_gating, use_bias=False, name="gating_W")(query)
    q_w = Lambda(lambda x: softmax(x, axis=1), output_shape=(params['query_term_maxlen'],))(q_w)
    q_w = Reshape((params["query_term_maxlen"],))(q_w)

    # combination of softmax(query term idf) and feed forward result per query term
    out_ = Dot(axes=[1, 1], name='s')([z, q_w])

    model = Model(inputs=[query, interactions], outputs=[out_])

    return model

# from https://github.com/faneshion/MatchZoo/blob/master/matchzoo/losses/rank_losses.py
from keras.backend import tf
from keras.losses import *
from keras.layers import Lambda
# y_true is IGNORED (!), you don't have to set a label to train (?)
# y_pred contains the complete batch (!)
#  -> the slicing splits the tensors in even and odd (pos and negative from the input)
#  -> VERY IMPORTANT: The input data must not be shuffled !! shuffle = False

def rank_hinge_loss(y_true, y_pred):

    y_pos = Lambda(lambda a: a[::2, :], output_shape= (1,))(y_pred)
    y_neg = Lambda(lambda a: a[1::2, :], output_shape= (1,))(y_pred)
    
    loss = K.maximum(0., 1. + y_neg - y_pos)
    return K.mean(loss)

#du coup pour l'entrainement dans le batch il faut aligner des paires de doc de la forme 
#positif-négatif-positif-négatif... etc

In [129]:
params = {
    "hidden_sizes": [5, 32, 1],
    "hist_size": 5,
    "query_term_maxlen": 4,
    "embedding_size": 300
}

drmm = build_keras_model(params)
drmm.compile(loss=rank_hinge_loss, optimizer='adam')
drmm.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
interactions (InputLayer)       (None, 4, 5)         0                                            
__________________________________________________________________________________________________
dense_layer_0 (Dense)           (None, 4, 5)         30          interactions[0][0]               
__________________________________________________________________________________________________
activation_of_layer_0 (Activati (None, 4, 5)         0           dense_layer_0[0][0]              
__________________________________________________________________________________________________
dense_layer_1 (Dense)           (None, 4, 32)        192         activation_of_layer_0[0][0]      
__________________________________________________________________________________________________
activation

In [133]:
from ann_visualizer.visualize import ann_viz

In [134]:
ann_viz(drmm, title="Artificial Neural network - Model Visualization")

ValueError: invalid literal for int() with base 10: ''